
---

# 📘 Integration with LangGraph

LangGraph is a **stateful, multi-agent framework** built on LangChain.
You can wrap LangGraph workflows as FastAPI endpoints to run them via HTTP.

---

## ✅ Step 1: Install Requirements

```bash
pip install fastapi uvicorn langgraph langchain openai
```

Set your OpenAI key (or use `.env`):

```bash
export OPENAI_API_KEY=sk-xxxxxx
```

---

## ✅ Step 2: Create a Simple LangGraph

This example builds a **two-node graph**:

* One node handles input
* Second node returns an LLM response

```python
# graph_setup.py
from langgraph.graph import StateGraph
from langchain.schema import HumanMessage
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI()

# Define node
def call_llm(state):
    user_input = state["input"]
    response = llm([HumanMessage(content=user_input)])
    return {"output": response.content}

# Create graph
def build_graph():
    builder = StateGraph()
    builder.add_node("llm_node", call_llm)
    builder.set_entry_point("llm_node")
    return builder.compile()
```

---

## ✅ Step 3: Build FastAPI Server

```python
# main.py
from fastapi import FastAPI
from pydantic import BaseModel
from graph_setup import build_graph

app = FastAPI()
graph = build_graph()

class Prompt(BaseModel):
    message: str

@app.post("/graph/")
async def run_graph(prompt: Prompt):
    result = graph.invoke({"input": prompt.message})
    return {"response": result["output"]}
```

---

## ✅ Step 4: Run FastAPI

```bash
uvicorn main:app --reload
```

Test endpoint:
`POST http://localhost:8000/graph/`

---

## ✅ Step 5: Test via Python

```python
import requests

res = requests.post("http://localhost:8000/graph/", json={"message": "What is LangGraph?"})
print(res.json())
```

---

## ✅ Summary Table

| Component    | Role                              |
| ------------ | --------------------------------- |
| `StateGraph` | LangGraph builder                 |
| `call_llm`   | Node function using LangChain LLM |
| `/graph/`    | FastAPI endpoint                  |
| `ChatOpenAI` | Handles LLM logic                 |

---

## 🧠 Optional Enhancements

* Add **multiple nodes** for decision logic
* Use **tools or memory** inside LangGraph nodes
* Stream output from FastAPI using `StreamingResponse`

---
